# What?
- plan implementation of the solution
- develop and deploy
- measure/monitor, learn, train
# How?
- design a solution (IT or not)
# Why?
- understand why you need this solution
- define the need/problem you are trying to address

# 월간특가 분석
1. (promo) pid 매출 변화
    - (bef promo pid Qty & Rev) vs. (dur promo pid Qty & Rev)
2. (promo) pid 가격 민감도
    - $$\frac{bef\,promo\,pid\,Rev}{bef\,promo\,pid_cate\,Rev}\;vs\;\frac{dur\,promo\,pid\,Rev}{dur\,promo\,pid_cate\,Rev}$$
    - 할인 -> 매출 비율이 높아진다.
    - 시사점 : 매출 비율 변화가 없다면 다음과 같은 추론이 가능하다.
        - pid_cate가 가격 민감도가 낮았다.
        - 대체 pid가 많았다(용중크 변화에 따른 pid).
        - pid 할인율이 낮았다.
3. (promo) pid_cate 경쟁력
    - $$\frac{잔존고객}{유입고객}$$
    - 기존고객
        - (promo - 1개월 ~ promo 시작) 中 pid_cate를 구매한 sid
    - 유입고객
        - 기존고객을 제외한 고객 中 pid를 구매한 sid
        - (promo - 1개월)까지 가입한 sid
        - (promo - 1개월 ~ promo 시작) 中 정식주문이 있는 sid
    - 잔존고객
        - 유입고객 中 (promo 종료 + 1개월) 內 pid_cate를 구매한 고객
    - 시사점 : pid_cate 잔존률을 비교하면 다음과 같은 추론이 가능하다.
        - 잔존률이 높다. -> pid_cate가 이미 경쟁력이 있다. -> 월간특가가 굳이 필요하지 않은 pid_cate다.
        - 잔존률이 높다. -> 월간특가를 (홍보측면에선) 잘했다.
0. 기타
    - who? : all orders data from (promo - 1개월) to (promo + 1개월)
    - 주의점 : bef promo 기간 = dur promo 기간(promo가 조기 종료되는 pid가 있기 때문)


In [26]:
# 1. 환경을 설정한다.
from datetime import date, timedelta

import calendar
import mysql.connector
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns



pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 500)
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
rc('font', family=font_name)

In [34]:
# 2. 필요한 데이터를 가져온다.

# 2-1. promo 데이터를 가져온다.
# promo = pd.read_excel('./data/promo_21-02.xlsx')
# promo = pd.read_excel('./data/promo_21-03.xlsx')
# promo = pd.read_excel('./data/promo_21-04.xlsx')
# promo = pd.read_excel('./data/promo_21-05.xlsx')
promo = pd.read_excel('./data/promo_21-06.xlsx')
date_promo_start = date(year=2021, month=6, day=1) # promo 시작일
days_in_month = timedelta(calendar.monthrange(date_promo_start.year, date_promo_start.month)[1]) # promo월에 해당하는 days 수
date_promo_end = date_promo_start + days_in_month # promo 종료일

# 2-2. db에서 데이터를 가져온다.
cnx = mysql.connector.connect(**connection_setting)
# cursor(commander) 객체를 가져온다.
cursor = cnx.cursor()

# query를 선언한다.
# 주문 <- from (promo - 2개월) to (promo + 1개월)
# 가입 <- from (promo - 1개월)

# qry에서 대상이 되는 기간을 정의한다.
date_start = date_promo_start - pd.Timedelta(days=60) # promo - 2개월
date_end = date_promo_end + pd.Timedelta(days=30) # promo + 1개월
date_constraint = date_promo_start - pd.Timedelta(days=30) # 가입일 : promo - 1개월

qry = f'''
SELECT
	ro.*,
	xms.product_cate,
	xms.rawname
FROM
	ople.rest_order ro
JOIN ople.rest_store rs ON
	ro.sid_id = rs.id
JOIN prd_master2.xprd_master_source xms ON
	ro.pid_id = xms.pid
WHERE
	ro.create_date BETWEEN '{date_start}' AND '{date_end}'
    AND ro.is_sid_temp = 0
    AND ro.status = 'Done'
    AND rs.is_statistics = 1
    AND rs.is_test = 0
    AND rs.create_date < '{date_constraint}'
'''

# data frame으로 만든다.
df_ori = pd.read_sql(qry, cnx)
# db 연결을 닫는다.
cnx.close()

# 3. data pre-processing
# 작업용 df를 선언한다.
df = df_ori.copy()

df = df[[
    'create_date',
    'pid_id',
    'count',
    'orders_id',
    'price',
    'sid_id',
    'price_sum',
    'product_cate',
    'rawname'
    ]]

# dt.normalize()
df['create_date'] = df.create_date.dt.normalize()
# create_date <- order_date로 컬럼명을 변경한다.
df.rename({'create_date':'order_date'}, axis=1, inplace=True)

In [35]:
# 4. func
# 변수를 선언한다.
list_pid_ori = list()
list_cate = list()
list_name = list()

list_date_promo_start = list()
list_date_promo_end = list()
list_date_promo_duration = list()
list_date_prev_end = list()
list_date_prev_start = list()
list_date_next_start = list()
list_date_next_end = list()

list_qty_promo = list()
list_qty_prev = list()
list_qty_next = list()

list_rev_promo = list()
list_rev_prev = list()
list_rev_next = list()

list_rev_promo_cate = list()
list_rev_prev_cate = list()
list_rev_next_cate = list()

list_sid_ori_count = list()
list_sid_new_count = list()
list_sid_left_count = list()

from tqdm import tqdm # tqdm <- 연산 시간을 확인할 수 있다.
# unique한 기존 pid 값이 나올 때마다 for문을 진행한다.
for pid_ori in tqdm(promo['기존 pid'].drop_duplicates()):
    list_pid_ori.append(pid_ori)
    pid_promo = promo[promo['기존 pid']==pid_ori]['프로모션용 pid']\
            .tolist() # pid_ori와 달리 pid_promo은 따로 list에 넣어준다.
                                # <- pid_promo은 한 개의 pid당 1~3개씩 있기 때문이다.
    try:
        cate = df[df['pid_id']==pid_ori]['product_cate'].values[0]
#         cate = df[df['pid_id']==pid_ori]['product_cate'].iloc[0]
        list_cate.append(cate)
        name = df[df['pid_id']==pid_ori]['rawname'].values[0]
#         name = df[df['pid_id']==pid_ori]['rawname'].iloc[0]
        list_name.append(name)
    #     print(pid_ori, pid_promo, cate, name, sep='\n')

        # 기간을 정의한다. <- promo excel sheet를 참조한다.
        # promo 기간
        date_promo_start = promo[promo['기존 pid']==pid_ori]['시작일'].iloc[0]
        list_date_promo_start.append(date_promo_start)
        date_promo_end = promo[promo['기존 pid']==pid_ori]['종료일'].iloc[0] # 종료일을 포함해야 하는지 확인해야 한다.
        list_date_promo_end.append(date_promo_end)
        date_promo_duration = date_promo_end - date_promo_start + pd.Timedelta(days=1)
        list_date_promo_duration.append(date_promo_duration)
        date_prev_end = date_promo_start
        list_date_prev_end.append(date_prev_end)
        date_prev_start = date_prev_end - date_promo_duration
        list_date_prev_start.append(date_prev_start)
        date_next_start = date_promo_end
        list_date_next_start.append(date_next_start)
        date_next_end = date_next_start + date_promo_duration
        list_date_next_end.append(date_next_end)

        # 4-1. func1 : (promo) pid 매출 변화
        # prev quantity
        qty_prev = df[
            (df['order_date']>=date_prev_start)
            & (df['order_date']<=date_prev_end)
            & (df['pid_id']==pid_ori)
        ]['count'].sum()
        list_qty_prev.append(qty_prev)
        # promo quantity
        qty_promo = df[
            (df['order_date']>=date_promo_start)
            & (df['order_date']<=date_promo_end)
            & (df['pid_id'].isin(pid_promo))
        ]['count'].sum()
        list_qty_promo.append(qty_promo)
        # next quantity
        qty_next = df[
            (df['order_date']>date_next_start)
            & (df['order_date']<=date_next_end)
            & (df['pid_id']==pid_ori)
        ]['count'].sum()
        list_qty_next.append(qty_next)

        # prev revenue
        rev_prev = df[
            (df['order_date']>=date_prev_start)
            & (df['order_date']<date_prev_end)
            & (df['pid_id']==pid_ori)
        ]['price_sum'].sum()
        list_rev_prev.append(rev_prev)
        # promo revenue
        rev_promo = df[
            (df['order_date']>=date_promo_start)
            & (df['order_date']<=date_promo_end)
            & (df['pid_id'].isin(pid_promo))
        ]['price_sum'].sum()
        list_rev_promo.append(rev_promo)
        # next quantity
        rev_next = df[
            (df['order_date']>date_next_start)
            & (df['order_date']<=date_next_end)
            & (df['pid_id']==pid_ori)
        ]['price_sum'].sum()
        list_rev_next.append(rev_next)


        # 4-2. func2 : pid 가격 민감도
        # prev cate revenue
        rev_prev_cate = df[
            (df['order_date']>=date_prev_start)
            & (df['order_date']<date_prev_end)
            & (df['product_cate']==cate)
            ]['price_sum'].sum()
        list_rev_prev_cate.append(rev_prev_cate)
        # promo cate revenue
        rev_promo_cate = df[
            (df['order_date']>=date_promo_start)
            & (df['order_date']<=date_promo_end)
            & (df['product_cate']==cate)
            ]['price_sum'].sum()
        list_rev_promo_cate.append(rev_promo_cate)
        # next cate revenue
        rev_next_cate = df[
            (df['order_date']>date_next_start)
            & (df['order_date']<=date_next_end)
            & (df['product_cate']==cate)
            ]['price_sum'].sum()
        list_rev_next_cate.append(rev_next_cate)


        # 4-3. func3 : pid_cate 경쟁력
        # 기존고객 : promo 전(from - 1개월)에도 이미 pid_cate를 구매한 sid
        list_sid_ori = df[
            (df['order_date']>=date_prev_start)
            & (df['order_date']<date_prev_end)
            & (df['product_cate']==cate)
        ]['sid_id'].drop_duplicates().tolist()
        list_sid_ori_count.append(len(list_sid_ori))
        # 유입고객 : 기존고객을 제외한 고객 중에서 pid를 구매한 sid
        # <- promo 전에는 pid_cate를 구매한 적이 없다.
        list_sid_new = df[
            (df['order_date']>=date_promo_start)
            & (df['order_date']<=date_promo_end)
            & (df['pid_id'].isin(pid_promo))
            & (~df['sid_id'].isin(list_sid_ori))
        ]['sid_id'].drop_duplicates().tolist()
        list_sid_new_count.append(len(list_sid_new))
        # 잔존고객 : promo 후(from + 1개월)에도 pid_cate를 구매하는 sid
        list_sid_left = df[
            (df['order_date']>date_next_start)
            & (df['order_date']<=date_next_end)
            & (df['product_cate']==cate)
            & (df['sid_id'].isin(list_sid_new))
        ]['sid_id'].drop_duplicates().tolist()
        list_sid_left_count.append(len(list_sid_left))
        
    except:
        list_cate.append('None')
        list_name.append('None')
        list_date_promo_start.append('None')
        list_date_promo_end.append('None')
        list_date_promo_duration.append('None')
        list_date_prev_end.append('None')
        list_date_prev_start.append('None')
        list_date_next_start.append('None')
        list_date_next_end.append('None')
        list_qty_prev.append('None')
        list_qty_promo.append('None')
        list_qty_next.append('None')
        list_rev_prev.append('None')
        list_rev_promo.append('None')
        list_rev_next.append('None')
        list_rev_promo_cate.append('None')
        list_rev_prev_cate.append('None')
        list_rev_next_cate.append('None')
        list_sid_ori_count.append(len('None'))
        list_sid_new_count.append(len('None'))
        list_sid_left_count.append(len('None'))

100%|██████████| 27/27 [00:02<00:00, 11.91it/s]


In [36]:
# 5. 집계한다.
report = pd.DataFrame({
    'pid':list_pid_ori,
    'cate':list_cate,
    'date_promo_start':list_date_promo_start,
    'date_promo_end':list_date_promo_end,
    'date_promo_duration':list_date_promo_duration,
    'date_prev_start':list_date_prev_start,
    'date_prev_end':list_date_prev_end,
    'date_next_start':list_date_next_start,
    'date_next_end':list_date_next_end,
    'qty_prev':list_qty_prev,
    'qty_promo':list_qty_promo,
    'qty_next':list_qty_next,
    'rev_prev':list_rev_prev,
    'rev_promo':list_rev_promo,
    'rev_next':list_rev_next,
    'rev_prev_cate':list_rev_prev_cate,
    'rev_promo_cate':list_rev_promo_cate,
    'rev_next_cate':list_rev_next_cate,
    'sid_ori_count':list_sid_ori_count,
    'sid_new_count':list_sid_new_count,
    'sid_left_count':list_sid_left_count
})
# report.to_excel('./report/report_21-02.xlsx')
# report.to_excel('./report/report_21-03.xlsx')
# report.to_excel('./report/report_21-04.xlsx')
# report.to_excel('./report/report_21-05.xlsx')
report.to_excel('./report/report_21-06.xlsx')


In [65]:
# datetime 모듈 사용에 익숙해지기
import datetime
datetime.datetime(year=2021, month=2, day=2, hour=10, minute=12, second=0)

input = '2021-02-02'
d = input.split('-')
d = [int(x) for x in d]
d

[2021, 2, 2]

In [66]:
d1 = datetime.datetime(year=int(d[0]), month=int(d[1]), day=int(d[2]))
d2 = datetime.timedelta(days=30)
d1 + d2

datetime.datetime(2021, 3, 4, 0, 0)